# Utilisation de l'api SNCF

Ce notebook a pour but d'utiliser l'api SNCF afin de répondre a la question suivante : 

Dans quelle gare il y a le plus de retard de train en France.

Pour utiliser l'api nous avons besoin d'une clef API qui sera stocker dans un secret.


__Librairies :__

python-dotenv

In [ ]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json
from datetime import datetime

Je charge ma clé API dans une variable

In [ ]:
load_dotenv()
api_key = os.getenv("API_KEY")

In [ ]:
def requete_api(requete):
    reponse = requests.get(requete, auth=(api_key,""))
    reponse.status_code
    reponse_json = reponse.json()
    return reponse_json

In [ ]:
def ecrire_json(data,nom_fichier):
    with open(nom_fichier, "w") as fc:
        json.dump(data, fc)

In [ ]:
def main():
    pass

### Exemples

__Le détail des modes de transport couvert par l’API__

In [ ]:
requete = "https://api.sncf.com/v1/coverage/sncf/commercial_modes"
reponse = requete_api(requete)

Mon fichier JSON est organisé en plusieur clés. Je récupère toutes les clés.

In [ ]:
for key in reponse.keys():
    print(key)

Les données qui m'interesse sont stocké dans la clé commercial_modes, je récupère donc les données de cette clé.

Je peux ensuite les charger dans un dataframe pandas

In [ ]:
data = reponse.get("commercial_modes")
dataframe = pd.DataFrame(data)
dataframe.head()

__Itinéraires entre Paris et Lyon__

In [ ]:
requete = "https://api.sncf.com/v1/coverage/sncf/journeys?from=admin:fr:75056&to=admin:fr:69123&datetime=20240123T105527"
reponse = requete_api(requete)
ecrire_json(reponse,"reponse.json")
for key in reponse.keys():
    print(key)

### Exercice

Récupérer les informations sur le trajet entre Paris Gare de Lyon et Lyon Perrache le 17 novembre à 19h57

* Indice : utiliser la requête “journeys”

* Autre indice : le format de la date est AAAAMMJJTHHMMSS (Année, mois, jour, heure, minutes, secondes

Combien y a t-il d'arrêts entre ces deux gares ?

Combien de temps d'arrêt à chacune d'elles ? 

__Je recherche les stop_area :__

In [ ]:
#Dans la région sncf je recherche les places contenant "lyon"
requete = "https://api.sncf.com/v1/coverage/sncf/places?q=lyon"
reponse = requete_api(requete)
ecrire_json(reponse,"liste place.json")

In [ ]:
code_gare_paris_gare_de_lyon = "stop_area:SNCF:87686006"
code_gare_lyon_perrache = "stop_area:SNCF:87722025"
date = "20240124T195700" 
url_base = "https://api.sncf.com/v1/coverage/sncf/journeys?"


In [ ]:
requete = f"{url_base}from={code_gare_paris_gare_de_lyon}&to={code_gare_lyon_perrache}&datetime={date}"
reponse = requete_api(requete)
ecrire_json(reponse,"reponse.json")

In [ ]:
#Nombre de voyage proposé
voyages = reponse.get("journeys")
nb_voyage = len(voyages)
print(f"il y a {nb_voyage} proposé.s")

Je veux garder uniquement le voyage le plus rapide

In [ ]:
def voyage_rapide(journeys):
    '''Cette fonction parcours une liste de voyage et renvoie le voyage le plus rapide'''

    voyage_le_plus_rapide = None
    for voyage in voyages:
        if voyage["type"] == "fastest":
            voyage_le_plus_rapide = voyage
            break

    if voyage_le_plus_rapide is None:
        print("Pas de voyage fastest trouvé")
    else:
        print("voyage rapide trouvé")

    return voyage_le_plus_rapide

In [ ]:
def decode_datetime(date_codee):
    #On enleve le T du code
    date_decodee = datetime.strptime(date_codee,"%Y%m%dT%H%M%S")
    return date_decodee

In [ ]:
def liste_arrets(voyage):
    '''Cette fonction extrait les arrets d'un voyage et les présente dans un dataframe avec les heures de départ et d'arrivée'''

    nb_correspondance = voyage["nb_transfers"]
    print(f"Dans ce voyage il y a {nb_correspondance} correspondances")
    # Pour un voyage sans correspondances les sections sont organisé de la façon suivante :
    # type = crow_fly / type=public_transport / type = crow_fly
    # On garde la deuxième section
    # Les arrêts sont stocké sous la clé stop_date_times
    arrets = voyage["sections"][1]["stop_date_times"]
    nb_arrets = len(arrets)
    print(f"Dans ce voyage il y a {nb_arrets} arrets")

    #Je récupère mes données sous forme d'une liste
    donnees = []

    for arret in arrets:
        #J'ajoute a chaque element de ma liste un dictionnaire contenant des informations sur les arrêts
        donnees.append(dict(nom = arret["stop_point"]["name"],
                        arrive = decode_datetime(arret["arrival_date_time"]),
                        depart = decode_datetime(arret["departure_date_time"])))

    dataframe = pd.DataFrame(donnees)

    return dataframe


In [ ]:
voyage = voyage_rapide(voyages)
arrets = liste_arrets(voyage)
arrets.head()

__Question 2 :__

Vous êtes un peu pressé et vous avez peur de vous tromper en arrivant à la gare car d’autres TGV partent à peu près en même temps (à partir de 19h00) de la gare de Lyon.

* Si vous demandez à l’API, combien de résultats vous donne-t-elle ?

* Quels sont les horaires de départ de ces trains ?

* Parmi ces trains, combien de trains ont pour destination finale Lyon et qui partent le 17 novembre ?

Je veux tous les trains qui partent de gare de Lyon à 19h

In [ ]:
code_gare_paris_gare_de_lyon = "stop_area:SNCF:87686006"
date = "20240125T190000"
url_base = "https://api.sncf.com/v1/coverage/sncf"
network = "network:SNCF:OUI"
physical_mode = "physical_mode:LongDistanceTrain"

In [ ]:
requete_network = f"{url_base}/stop_areas/{code_gare_paris_gare_de_lyon}/networks/{network}/departures?from_datetime={date}"
requete_physical_mode =f"{url_base}/stop_areas/{code_gare_paris_gare_de_lyon}/physical_modes/{physical_mode}/departures?from_datetime={date}"
reponse = requete_api(requete_physical_mode)
ecrire_json(reponse,"exercice_2.json")

In [ ]:
data=[]
departs = reponse["departures"]

for depart in departs:
    
    heure_depart =decode_datetime(depart["stop_date_time"]["departure_date_time"])
    heure_depart_prevue = decode_datetime(depart["stop_date_time"]["base_departure_date_time"])


    data.append(dict(commercial_mode = depart["display_informations"]["commercial_mode"],
                    reseau = depart["display_informations"]["network"],
                    numero = depart["display_informations"]["headsign"],
                    direction = depart["display_informations"]["direction"],
                    heure_depart =decode_datetime(depart["stop_date_time"]["departure_date_time"]),
                    heure_depart_prevue = decode_datetime(depart["stop_date_time"]["base_departure_date_time"]),
                    delais=  heure_depart - heure_depart_prevue ))

dataframe = pd.DataFrame(data)
dataframe.head(20)

__Question 3:__

En fait, vous n’êtes plus très sûr de vouloir aller à Lyon. Mais bon maintenant vous êtes Gare de Lyon et il est 18h00.

* Combien de tgv partent entre 18h00 et 20h00 ?

* Lequel arrive le plus tôt à sa destination finale ?

In [ ]:
code_gare_paris_gare_de_lyon = "stop_area:SNCF:87686006"
date_debut = "20240125T180000"
date_fin = "20240125T200000"
url_base = "https://api.sncf.com/v1/coverage/sncf"
physical_mode = "physical_mode:LongDistanceTrain"

Je fais ma requete en filtrant sur le physical_mode et sur les datetime. J'augmente également le nombre de réponse à 20

In [22]:
requete_physical_mode =f"{url_base}/stop_areas/{code_gare_paris_gare_de_lyon}\
/physical_modes/{physical_mode}/departures?from_datetime={date_debut}&max_datetime={date_fin}&count=20&"
reponse = requete_api(requete_physical_mode)
ecrire_json(reponse,"exercice_3.json")

In [40]:
def departs_to_dataframe(liste_departs):
    '''Cette fonction parcours tous les départs d'une liste, en extrait des informations et retourne un dataframe'''

    data=[]
    for depart in liste_departs:
        heure_depart =decode_datetime(depart["stop_date_time"]["departure_date_time"])
        heure_depart_prevue = decode_datetime(depart["stop_date_time"]["base_departure_date_time"])


        data.append(dict(reseau = depart["display_informations"]["network"],
                        numero = depart["display_informations"]["headsign"],
                        direction = depart["display_informations"]["direction"],
                        heure_depart =decode_datetime(depart["stop_date_time"]["departure_date_time"]),
                        heure_depart_prevue = decode_datetime(depart["stop_date_time"]["base_departure_date_time"]),
                        delais=  heure_depart - heure_depart_prevue,
                        heure_arrivee = decode_datetime(depart["stop_date_time"]["arrival_date_time"])))
    
    dataframe=pd.DataFrame(data)
    
    return dataframe

In [30]:
departs = reponse["departures"]

dataframe = departs_to_dataframe(departs)
dataframe.head(5)

,commercial_mode,reseau,numero,direction,heure_depart,heure_depart_prevue,delais,heure_arrivee
0,TGV INOUI,TGV INOUI,917,Bourg-Saint-Maurice (Bourg-Saint-Maurice),2024-02-17 05:59:00,2024-02-17 05:59:00,0 days,2024-02-17 05:59:00
1,OUIGO,OUIGO,7829,Marseille Saint-Charles (Marseille),2024-02-17 06:03:00,2024-02-17 06:03:00,0 days,2024-02-17 06:03:00
2,TGV INOUI,TGV INOUI,705,Saint-Gervais-les-Bains - Le Fayet (Saint-Gerv...,2024-02-17 06:11:00,2024-02-17 06:11:00,0 days,2024-02-17 06:11:00
3,TGV Lyria,TGV Lyria,9761,Lausanne (Lausanne),2024-02-17 06:18:00,2024-02-17 06:18:00,0 days,2024-02-17 06:18:00
4,TGV INOUI,TGV INOUI,921,Bourg-Saint-Maurice (Bourg-Saint-Maurice),2024-02-17 06:22:00,2024-02-17 06:22:00,0 days,2024-02-17 06:22:00


On utilise pandas pour filtrer le dataframe et gardes train qui arrivent avant 20h

In [ ]:
limite_heure = "2024-01-25 20:00:00"
depart_avant_20h = dataframe.query("heure_depart < '2024-01-25 20:00:00'")
depart_avant_20h.tail()

In [ ]:
print(f"il y a {len(depart_avant_20h)} départs entre 18h et 20h")

In [ ]:
print("Le train qui arrive en premier est le suivant :")
depart_avant_20h.sort_values(by=["heure_arrivee"]).head(1)

__Question 4:__

On va essayer de voir jusqu’où on peut aller, en prenant des trains au départ de la Gare de Lyon :

* Quelles sont toutes les gares atteignables en partant le 17 novembre, sans faire de changement et sans partir après minuit ?

* Si on prend un de ces trains, jusqu’où peut-on aller, avec une correspondance, sans partir après 8h le lendemain matin ?

In [19]:
def tous_trajet(gare,heure_min,heure_max):

    base_url = "https://api.sncf.com/v1/coverage/sncf"
    physical_mode = "physical_mode:LongDistanceTrain"

    tous_les_departs = []
    heure_dernier_depart = heure_min
    heure_dernier_depart_decode = decode_datetime(heure_dernier_depart)
    heure_max_decode = decode_datetime(heure_max)


    while heure_dernier_depart_decode < heure_max_decode:

        requete =f"{base_url}/stop_areas/{gare}/physical_modes/{physical_mode}/departures?from_datetime={heure_dernier_depart}"
        reponse = requete_api(requete)
        departs = reponse["departures"]
        dernier_depart = departs[-1]
        heure_dernier_depart = dernier_depart["stop_date_time"]["base_departure_date_time"]
        heure_dernier_depart_decode = decode_datetime(heure_dernier_depart)
        #J'ajoute la liste des depart a ma liste de tous les depart
        # Je n'utilise pas apprend car sinon ajoute une liste dans une liste
        tous_les_departs += departs
    
    return tous_les_departs


In [ ]:
code_gare_paris_gare_de_lyon = "stop_area:SNCF:87686006"
date_debut = "20240217T050000"
date_fin = "20240217T235959"
url_base = "https://api.sncf.com/v1/coverage/sncf"

In [31]:
tous_trajets = tous_trajet(code_gare_paris_gare_de_lyon,date_debut,date_fin)
print(f" il y a {(len(tous_trajets))} trajets")

 il y a 140 trajets


In [41]:
mes_trajets = departs_to_dataframe(tous_trajets)
mes_trajets.head()

,reseau,numero,direction,heure_depart,heure_depart_prevue,delais,heure_arrivee
0,TGV INOUI,917,Bourg-Saint-Maurice (Bourg-Saint-Maurice),2024-02-17 05:59:00,2024-02-17 05:59:00,0 days,2024-02-17 05:59:00
1,OUIGO,7829,Marseille Saint-Charles (Marseille),2024-02-17 06:03:00,2024-02-17 06:03:00,0 days,2024-02-17 06:03:00
2,TGV INOUI,705,Saint-Gervais-les-Bains - Le Fayet (Saint-Gerv...,2024-02-17 06:11:00,2024-02-17 06:11:00,0 days,2024-02-17 06:11:00
3,TGV Lyria,9761,Lausanne (Lausanne),2024-02-17 06:18:00,2024-02-17 06:18:00,0 days,2024-02-17 06:18:00
4,TGV INOUI,921,Bourg-Saint-Maurice (Bourg-Saint-Maurice),2024-02-17 06:22:00,2024-02-17 06:22:00,0 days,2024-02-17 06:22:00


In [34]:
mes_trajets.tail()

,commercial_mode,reseau,numero,direction,heure_depart,heure_depart_prevue,delais,heure_arrivee
135,TGV INOUI,TGV INOUI,6129,Marseille Saint-Charles (Marseille),2024-02-17 19:38:00,2024-02-17 19:38:00,0 days,2024-02-17 19:38:00
136,OUIGO,OUIGO,7827,Marseille Saint-Charles (Marseille),2024-02-17 20:09:00,2024-02-17 20:09:00,0 days,2024-02-17 20:09:00
137,TGV Lyria,TGV Lyria,9789,Genève - Cornavin - CFF (Genève),2024-02-17 20:18:00,2024-02-17 20:18:00,0 days,2024-02-17 20:18:00
138,TGV INOUI,TGV INOUI,917,Bourg-Saint-Maurice (Bourg-Saint-Maurice),2024-02-18 05:50:00,2024-02-18 05:50:00,0 days,2024-02-18 05:50:00
139,OUIGO,OUIGO,7829,Marseille Saint-Charles (Marseille),2024-02-18 06:03:00,2024-02-18 06:03:00,0 days,2024-02-18 06:03:00


In [44]:
mes_trajets.query("delais != 0 days").head()

SyntaxError: invalid syntax (<unknown>, line 1)